In [ ]:
import ipyvuetify as v

from seeq_sysid.gui.setup import Setup
from seeq_sysid.gui.app_bar import AppBar
from seeq_sysid.gui.transfer_matrix import TransferMatrix
from seeq_sysid.gui.figure_table import FigureTable

from pandas import DataFrame
# from pandas import read_csv # For Local Run

from seeq_sysid.gui._backend import *
import urllib.parse as urlparse
from urllib.parse import parse_qs
from seeq import spy

import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML, clear_output, display, Javascript

jv = '''$('#header-container').hide();'''
Javascript(jv)


In [ ]:
%%javascript

if (document.addEventListener) {
  document.addEventListener('contextmenu', function(e) {
    e.preventDefault();
  }, false);
} else {
  document.attachEvent('oncontextmenu', function() {
      window.event.returnValue = false;
  });
}

In [ ]:
class App(v.App):
    def __init__(self):
        class_="d-flex justify-center flex-column mx-auto ma-4 pa-4"
        style_='width:100%'
        super().__init__(class_=class_,
                       style_=style_)
        
        
        self.worksheet_url = ''
    
        # Server Mode
        self.addon_worksheet = 'From ARX AddOn'
        self.workbook_id = None
        self.worksheet_url = None
        
        try:
            sdl_notebook_url = jupyter_notebook_url
            self.workbook_id, self.worksheet_id, self.workstep_id = get_workbook_worksheet_workstep_ids(sdl_notebook_url)
            self.worksheet_url = get_worksheet_url(sdl_notebook_url)
            self.signal_df, self.capsule_df, self.tags_df = pull_signals(self.worksheet_url)

        except:
            self.signal_df = DataFrame()
            self.capsule_df = DataFrame()
            self.tags_df = DataFrame()
        
        # Local Mode
        # self.signal_df = read_csv('signal_df.csv')
        # self.signal_df = self.signal_df.iloc[50:]
        # self.signal_df.set_index('Time', inplace=True)
        # self.capsule_df = read_csv('capsule_df.csv')
        # self.capsule_df.set_index('Time', inplace=True)
        # self.tags_df = DataFrame()
                
        self.window = v.Window(v_model=0, class_='mt-3')
        
        # Import Sheet
        self.import_sheet = Setup()
        self.import_sheet.set_data(self.signal_df.columns.to_list())
        self.import_window = v.WindowItem(children=[self.import_sheet], style_='width:100%', transition='none', reverse_transition='none')
        
        # Matrix Sheet x2
        self.matrix_sheet = TransferMatrix()
        self.matrix_window = v.WindowItem(children=[self.matrix_sheet], style_='width:100%', transition='none', reverse_transition='none')
        
        # Visualization Sheet
        self.visual_sheet = FigureTable()
        self.visual_window = v.WindowItem(children=[self.visual_sheet], style_='width:100%', transition='none', reverse_transition='none')
        
        # Set Windows
        self.window.children = [self.import_window, self.matrix_window, self.matrix_window, self.visual_window]
        
        
        self.app_bar = AppBar(title_list=['Import Data', 'Model Setup', 'Step Response', 'Visualization'])

        self.app_bar.title.children = [self.app_bar.title_list[self.window.v_model]]
        
        self.worksheet_url_box = self.app_bar.ham_menu.worksheet_url
        self.ok_url_dialog_btn = self.app_bar.ham_menu.ok_url_dialog_btn
        self.close_url_dialog_btn = self.app_bar.ham_menu.close_url_dialog_btn
        
        self.ok_url_dialog_btn.on_event('click',self.ok_url_action)
        self.close_url_dialog_btn.on_event('click',self.close_url_action)
        
        
        self.children = [self.app_bar, self.window]
        
        # Actions
        self.import_sheet.mv_select.on_event('change', self.update_panel)
        self.import_sheet.cv_select.on_event('change', self.update_panel)
        self.app_bar.back_btn.on_event('click', self.back_button_action)
        
        self.update_nav()
        self.update_panel()
        
    
    def update_nav(self):
        page_num = self.window.v_model
        
        self.app_bar.title.children = [self.app_bar.title_list[self.window.v_model]]
        
        if page_num == 0:
            self.app_bar.next_btn.on_event('click', self.import_sheet_next_action)
            self.app_bar.next_btn.children = ['Next >']
            self.app_bar.nav_btn.children = [self.app_bar.next_btn]  
            
        elif page_num == len(self.app_bar.title_list)-1:
            self.app_bar.nav_btn.children = [self.app_bar.back_btn]
            
        elif page_num == 2:
            self.app_bar.nav_btn.children = [self.app_bar.back_btn, self.app_bar.next_btn]
            self.app_bar.next_btn.on_event('click', self.step_sheet_next_action)
            
        elif page_num == 1:
            self.app_bar.next_btn.on_event('click', self.matrix_sheet_next_action)
            self.app_bar.nav_btn.children = [self.app_bar.back_btn, self.app_bar.next_btn]
            
            
    def update_panel(self, *_):
        mv_select = self.import_sheet.mv_select
        cv_select = self.import_sheet.cv_select

        mv_select.items = [item for item in self.import_sheet.signal if item not in cv_select.v_model]
        cv_select.items = [item for item in self.import_sheet.signal if item not in mv_select.v_model]

        if len(mv_select.v_model) > 0 and len(cv_select.v_model) > 0:
            self.app_bar.next_btn.disabled = False

        else:
            self.app_bar.next_btn.disabled = True  
            
    def import_sheet_next_action(self, item, *args):        
        mv, cv = self.import_sheet.get_data()
        
        item.loading = True
        self.matrix_sheet = TransferMatrix()
        self.matrix_sheet.create_matrix(df=self.signal_df[mv + cv], mv=mv, cv=cv)
        self.matrix_window.children = [self.matrix_sheet]
        self.window.children = [self.import_window, self.matrix_window, self.matrix_window, self.visual_window]
        
        item.loading = False
        self.window.v_model = 1
        self.update_nav()
            
    def matrix_sheet_next_action(self, item, *args):        
        item.loading = True

        self.matrix_sheet.run()
    
        item.loading = False
        self.window.v_model = 2
        self.update_nav()
        
    def step_sheet_next_action(self, item, *args):
        item.loading = True
        self.visual_sheet = FigureTable()
        self.visual_sheet.create(self.matrix_sheet.train_df, self.matrix_sheet.train_df)

        self.window.v_model = 3
        self.update_nav()

        self.visual_window.children = [self.visual_sheet]

        item.loading = False
        
    def back_button_action(self, *args):
        self.window.v_model += -1
        self.update_nav()
        
    def run(self):
        clear_output()
        display(HTML("""<style>.container {width:100% !important}</style>"""))
        return self


    # Load Worksheet Dialog Actions
    def load_worksheet(self):
        worksheet_url = self.worksheet_url_box.v_model
        if worksheet_url:
            sdl_notebook_url = f'{spy.utils.get_data_lab_project_url()}/dummy.ipynb?workbookId={spy.utils.get_workbook_id_from_url(worksheet_url)}&worksheetId={spy.utils.get_worksheet_id_from_url(worksheet_url)}'
            self.workbook_id, self.worksheet_id, self.workstep_id = get_workbook_worksheet_workstep_ids(sdl_notebook_url)
            self.worksheet_url = get_worksheet_url(sdl_notebook_url)
            self.signal_df, self.capsule_df, self.tags_df = pull_signals(self.worksheet_url)
            
        self.window.v_model = 0
        
        # Set Imported Data
        self.import_sheet.set_data(self.signal_df.columns.to_list())

        self.worksheet_url_box = self.app_bar.ham_menu.worksheet_url
        self.ok_url_dialog_btn = self.app_bar.ham_menu.ok_url_dialog_btn
        self.close_url_dialog_btn = self.app_bar.ham_menu.close_url_dialog_btn
        
        self.ok_url_dialog_btn.on_event('click',self.ok_url_action)
        self.close_url_dialog_btn.on_event('click',self.close_url_action)

        clear_output()
        display(HTML("""<style>.container {width:100% !important}</style>"""))
        self.children = [self.app_bar, self.window]
        self.run()
            
    
    def close_url_action(self, *args):
        self.worksheet_url_box.v_model = self.worksheet_url
        self.app_bar.ham_menu.url_dialog.v_model = None

    def ok_url_action(self, *args):
        self.ok_url_dialog_btn.loading = True
        self.worksheet_url = self.worksheet_url_box.v_model
        self.load_worksheet()
        self.worksheet_url_box.v_model = self.worksheet_url
        self.ok_url_dialog_btn.loading = False
        self.app_bar.ham_menu.url_dialog.v_model = None
        
    def push_data(self, *_):
        signal_df = self.create_dataset()
        self.all_results.set_index(signal_df.index, inplace=True)
        self.all_results[self.model.cv] = signal_df[self.model.cv]

        push_signal(self.all_results, self.workbook_id, self.addon_worksheet)


In [ ]:
a = App()
a.run()